In [32]:
import pandas as pd
import numpy as np

In [33]:
file_path = '/Users/diana/Desktop/isear-train.xlsx'
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

conda install openpyxl

In [34]:
df.head ()

,Emotions,Text
0,sadness,Losing my girlfriend who made an end to our re...
1,disgust,[ No response.]
2,fear,Staying alone in a dark place.
3,shame,When I failed grade 7.
4,anger,I am a teacher in arts and crafts (boys from 1...


In [35]:
total_count = df['Emotions'].value_counts().sum()
print (total_count)

5366


In [36]:
df['Emotions'].value_counts()


Emotions
sadness    775
fear       770
guilt      767
shame      765
anger      764
joy        764
disgust    761
Name: count, dtype: int64

In [37]:
print (df['Emotions'])
print (df['Emotions'].shape)
print (df['Emotions'].values)

0       sadness
1       disgust
2          fear
3         shame
4         anger
         ...   
5361       fear
5362      anger
5363    sadness
5364    disgust
5365       fear
Name: Emotions, Length: 5366, dtype: object
(5366,)
['sadness' 'disgust' 'fear' ... 'sadness' 'disgust' 'fear']


In [38]:
emotion_labels = list (set (df['Emotions'].values))

In [39]:


def tokenize(text):
    translation_table = str.maketrans({c: f' {c} ' if not c.isalnum() else c for c in set(text)})  # creates translation
    # table(dictionary) with the built-in function maketrans, set(text) makes an unordered collection of unique
    # elements through set comprehension, a concise way to create sets
    tokenized_text = text.translate(translation_table)  # uses translation table to add whitespace around special
    # characters and punctuation
    return tokenized_text.strip().lower().split()  # split-tokens split on space, lower-tokens made all lowercase,
    # strip-and leading or trailing whitespaces are removed from string


#extract dict
text = ''.join(df['Text'].astype(str))
tokenized_text = tokenize(text) #all tokens, including repeating
#print (tokenized_text)
vocab = set(tokenized_text)
#print(vocab)

tokenized_text = []
for index, sentence in enumerate(df['Text']):
    emotion_label = df['Emotions'][index]
    sentence = tokenize(sentence)
    tokenized_text.append (sentence)
    #print(emotion_label, sentence)
#print (tokenized_text)


In [41]:
from math import log

collection = [token for instance in tokenized_text for token in instance]

#tf-idf

def calculate_tfidf(token, document):
    tf = np.log(document.count(token) / len(document)) if token in document else 0 # 1+log(#oftimestokenindoc/total#termsindoc)
    idf = log (1 + (len(tokenized_text) / collection.count(token))) # log(#ofdocs/#oftimestokenappearsincollection)
    tfidf = tf * idf
    return tfidf

tfidf_scores_list = []
for sentence in tokenized_text:
        tfidf_scores = {token: calculate_tfidf(token, sentence) for token in set(sentence)}
        tfidf_scores_list.append (tfidf_scores)
#print (tfidf_scores_list)


In [42]:
from scipy.sparse import csr_matrix

# Construct a vocabulary list from the tfidf_scores_list
vocab = list(set(token for scores in tfidf_scores_list for token in scores.keys()))

# Construct a mapping from term to column index
term_to_col = {term: idx for idx, term in enumerate(vocab)}

# Convert the list of TF-IDF scores into a CSR sparse matrix
n_docs = len(tfidf_scores_list)
n_terms = len(vocab)
rows = []
cols = []
data = []
for doc_idx, scores in enumerate(tfidf_scores_list):
    for term, tfidf in scores.items():
        col_idx = term_to_col[term]
        rows.append(doc_idx)
        cols.append(col_idx)
        data.append(tfidf)

X_tfidf_sparse = csr_matrix((data, (rows, cols)), shape=(n_docs, n_terms))

print (X_tfidf_sparse)

  (0, 103)	-3.390157501177657
  (0, 150)	-11.678476673570804
  (0, 449)	-1.2184716044643313
  (0, 456)	-30.28447013048586
  (0, 500)	-3.45916834149466
  (0, 793)	-22.541516563613403
  (0, 1011)	-15.107739666568516
  (0, 1209)	-11.54723730712528
  (0, 1635)	-3.3738515713905723
  (0, 1744)	-1.5097589425443292
  (0, 1796)	-14.552470884076424
  (0, 1845)	-20.517164637414933
  (0, 1987)	-4.462081057794252
  (0, 2744)	-26.411680969086532
  (0, 2896)	-22.541516563613403
  (0, 3007)	-30.28447013048586
  (0, 3362)	-30.28447013048586
  (0, 3419)	-14.323756562375893
  (0, 3809)	-13.073665663494369
  (0, 3963)	-17.39031069764966
  (0, 4143)	-9.821200355624516
  (0, 5473)	-5.252881165518687
  (0, 5533)	-7.6915332074665645
  (0, 5788)	-6.78703851313937
  (0, 5883)	-9.017159890762102
  :	:
  (5365, 449)	-1.7548752030203894
  (5365, 500)	-3.4298842044059197
  (5365, 770)	-12.88804538355454
  (5365, 853)	-13.790087239012053
  (5365, 986)	-30.028092155374704
  (5365, 1231)	-8.807941061382586
  (5365, 16

In [43]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.sparse import csr_matrix

# Convert the list of tokenized sentences into a list of strings
"""corpus = [' '.join(sentence) for sentence in tokenized_text]

# Create a CountVectorizer to convert the corpus into a document-term matrix
count_vectorizer = CountVectorizer(vocabulary=vocab)
X_count = count_vectorizer.fit_transform(corpus)

# Create a TfidfTransformer to compute the TF-IDF scores
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_count)

# Convert the TF-IDF matrix to CSR format if needed
if not isinstance(X_tfidf, csr_matrix):
    X_tfidf = X_tfidf.tocsr()"""

#print (corpus)
#print(X_tfidf)

y = df['Emotions'].values


In [ ]:
tfidf_df = pd.DataFrame(X_tfidf)

tfidf_df.to_csv('tfidf_representations.csv', index=False)

tfidf_df.to_excel('tfidf_representations.xlsx', index=False)

"tfidf_df = pd.DataFrame(X_tfidf)\n\ntfidf_df.to_csv('tfidf_representations.csv', index=False)\n\ntfidf_df.to_excel('tfidf_representations.xlsx', index=False)"

In [44]:
%store X_tfidf
%store X_tfidf_sparse
%store y

Stored 'X_tfidf' (csr_matrix)
Stored 'X_tfidf_sparse' (csr_matrix)
Stored 'y' (ndarray)
